What about doing a brute froce "smt" egraph using grind instead?

Dependent Hmaps in lean
https://xavierleroy.org/bibrefs/Appel-Leroy-extensional.html Andrew W. Appel and Xavier Leroy. Efficient extensional binary tries. Journal of Automated Reasoning, 67:article 8, 2023.

Talia

Congruence closure Leo

GATs

Bildingmaier

https://arxiv.org/pdf/2505.08496  Weighted Rewriting: Semiring Semantics for
Abstract Reduction Systems
Provenance / semiring semantics is an interesting variant of proof objects. Having proof objects for ring like things makes proof theory truncate naturally (free resolution)
Scallop. Max plus.


Datalog for tensor equations. Let the compiler do the rearrangement. Linear Relations. Born approximation. Dyson. 

Take the miniparser

https://www.philipzucker.com/telescope_tries/
https://github.com/true-grue/python-dsls/blob/main/datalog/datalog_test.py
`x : A`  typeof form.
I guess I was using `x in A` hmm.
`x : A`

https://github.com/true-grue/python-dsls/tree/main/pywasm This is pretty sick.


Atomic type theory.
`x : Bool |- Bool` ===> `true : Bool, false : Bool, x : Bool |- Bool`  `[[Bool]] = {true, false, x}` If I have function symbols things get out of control. Unless I can do an EPR like thing of having the sorts stratified. Records are ok. Relations are ok. The EPR of type theory? Or a telescoping view on EPR?
exists c1 c2 c3 c4, forall x, p(x) forall y, q(x,y) => forall =>  

Intutitionstic propsotional logic is kind of "atomic type theory". But you swallow the poiuson pill of lambda / ->  type constructor which kind of screws you.

I guess I want to think about a subset of GATs that are so limited they are obviously decidable and are a union find variant.

Atmoic equational logic
Atomic predicate logic ~ propositional logic
Atomic inequality logic
Atomic contextual logic ~ atomic type theory? Telescope vs sequent form.

If all my union find variants have a "logic" coilumn in their table:
Atomic groupoid logic
Atomic primitive logic
Atomic multiset logic   ~~~ Linear logic? Bunched?
Atomic polynomial logic? = grobner



types "are" sets of terms in the syntactical "model". The set of terms that can be given that type. This is not 




In [28]:
n1: int

In [ ]:
def to_query(has_type : str):
    x = ast.parse(has_type, mode="func_type")
    body = x.argtypes
    FROMS = []
    WHERES = []
    env = {}
    for bind_num, binding in enumerate(body):
        match binding:
            case ast.Compare(left=ast.Name(id=var_name), ops=[ast.In()], comparators=[ast.Call(func=ast.Name(id=type_name), args=args, keywords=[])]):
                row = var_name + "_row"
                FROMS.append(f"{type_name} as {row}")
                env[var_name] = row + ".elem"
                for n,arg in enumerate(args):
                    WHERES.append(f"{env[arg.id]} = {row}.x{n}")
            case _:
                raise ValueError(f"Ill formed binding", ast.unparse(binding), ast.dump(binding))
            
    match x.returns:
        case ast.Compare(left=ast.Name(id=t), ops=[ast.In()], comparators=[ast.Call(func=ast.Name(id=type_name), args=args, keywords=[])]):
            selects = [f"{env[arg.id]}" for arg in args] + [env[t]]
            return f"INSERT INTO {type_name} SELECT {", ".join(selects)}" + "\nFROM\n" + ",\n". join(FROMS) + "\nWHERE\n" + " AND\n".join(WHERES)
        case _ :
            raise ValueError("Ill formed return type", ast.unparse(x.returns), ast.dump(x.returns))



In [22]:
from kdrag.solvers.egraph import EGraph
from kdrag.all import *
from dataclasses import dataclass
@dataclass
class DEGraph(EGraph):
    types : dict[int, set[int]]
    def __init__(self):
        super().__init__()
        self.types = {}
    def add_type(self, S : smt.ExprRef):
        # get held sort
        assert smt.is_array(S)
        self.add_term(S)
        Sid = self.find(S)
        if Sid not in self.types:
            tids = set()
            print(S)
            T = S.domain()
            for tid in self.roots[T]:
                with self.solver:
                    self.solver.add(smt.Not(S[self.terms[tid]]))
                    if self.solver.check() == smt.unsat:
                        tids.add(tid)
            self.types[Sid] = tids
    def assign_type(self, t, A):
        tid, Aid = self.find(t), self.find(A)
        self.types[Aid].add(tid)
        





E = DEGraph()
Vec = smt.Function("Vec", smt.IntSort(), smt.SetSort(smt.SeqSort(smt.BoolSort())))
Vec(3).sort().domain()
#n = smt.Int("n")
#l = smt.
Fin = smt.Function("Fin", smt.IntSort(), smt.SetSort(smt.IntSort()))
E.add_type(Fin(0))
E.add_type(Fin(1))
E.assign_type(smt.IntVal(0), Fin(1))
E


Fin(0)
Fin(1)


DEGraph(roots=defaultdict(<class 'set'>, {Array(Int, Bool): {17, 39}, Int: {16, 35}}), terms={17: Fin(0), 16: 0, 39: Fin(1), 35: 1}, uf={}, solver=[], reasons={}, types={17: set(), 39: {16}})

In [1]:
from kdrag.solvers.egraph import EGraph
from kdrag.all import *
class DependentEGraph(EGraph):
    types : dict[smt.ExprRef, set[smt.ExprRef]]
    def is_type(self, A: smt.ExprRef) -> bool:
        return self.find(A) in self.sets
    def make_type(self, expr: smt.ExprRef) -> None:
        if expr not in self.sets:
            self.sets[expr] = set()
    def has_type(self, t : smt.ExprRef, A : smt.ExprRef) -> bool:
        A = self.find(A)
        if self.is_type(A):
            return self.find(t) in self.sets[A]
        else:
            return False
    def assign_type(self, t: smt.ExprRef, A: smt.ExprRef) -> None:
        A = self.find(A)
        if not self.is_type(A):
            raise ValueError(f"Cannot assign type {A} to {t}, {A} is not a type")
        t = self.find(t)
        self.sets[A].add(t)
    def is_def_eq(self, t1: smt.ExprRef, t2: smt.ExprRef, A : smt.ExprRef) -> bool:
        return self.has_type(t1, A) and self.has_type(t2, A) and self.find(t1).eq(self.find(t2))
    def assert_def_eq(self, t1: smt.ExprRef, t2: smt.ExprRef, A: smt.ExprRef) -> None:
        A = self.find(A)
        self.assign_type(t1, A)
        self.assign_type(t2, A)
        self.union(t1, t2)
    def rebuild(self):
        super().rebuild()
        newsets = defaultdict(set)
        for k, v in self.sets.items():
            newsets[self.find(k)] |= {self.find(x) for x in v}
        self.sets = newsets